<a href="https://colab.research.google.com/github/MartynaPlomecka/AI4Health/blob/main/PCA_on_spectras_on_fixed_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy.io
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
data = scipy.io.loadmat('x.mat')
columns = np.asarray([['Electrode %d - %dHz'%(i+1, j+1)] for i in range(data['x'].shape[1]) for j in range(data['x'].shape[2])])
data['x'].shape

(2042, 105, 79)

In [ ]:
labels = pd.read_csv("table_withlabels.csv")
foof = pd.read_csv("foof2features.csv")
foof

,C1,C2,C3
0,NDARAA075AMK,0.986272,1.825774
1,NDARAA112DMH,1.486650,1.888544
2,NDARAA117NEJ,1.593155,2.095749
3,NDARAA947ZG5,0.703331,1.724831
4,NDARAA948VFH,0.918020,1.749441
...,...,...,...
2037,NDARZN277NR6,1.351549,1.996940
2038,NDARZN578YDP,1.380795,2.036327
2039,NDARZN610GTY,0.339229,1.050644
2040,NDARZN677EYE,0.781225,1.470061


In [ ]:
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df.columns = columns
df['IDs'] = foof['C1']

In [ ]:
df2 = pd.merge(df, labels[['label', 'IDs']], on='IDs', how='inner')

In [ ]:
print(df2['label'].value_counts())

Other Neurodevelopmental Disorders    492
ADHD-Inattentive Type                 388
ADHD-Combined Type                    376
Anxiety Disorders                     241
No Diagnosis Given                    203
Depressive Disorders                   85
Name: label, dtype: int64


In [ ]:
X = df2[df2.columns.difference(['IDs', 'label'])]
y = df2['label']


In [ ]:
# dimensionality reduction
pca = PCA(.95) # 95% variance retained
pca.fit(X)

# transform data
X_pca = pca.transform(X)
X_pca.shape

(1785, 132)

In [ ]:
n = pca.components_.shape[0]
pc_components = [np.abs(pca.components_[i]).argmax() for i in range(n)]
pc_columns = [X.columns[pc_components[i]] for i in range(n)]

X = pd.DataFrame(X_pca) 
X.columns = pc_columns
X

,"(Electrode 63 - 20Hz,)","(Electrode 56 - 16Hz,)","(Electrode 70 - 20Hz,)","(Electrode 75 - 23Hz,)","(Electrode 19 - 46Hz,)","(Electrode 86 - 59Hz,)","(Electrode 60 - 19Hz,)","(Electrode 60 - 16Hz,)","(Electrode 68 - 38Hz,)","(Electrode 57 - 18Hz,)","(Electrode 12 - 19Hz,)","(Electrode 61 - 17Hz,)","(Electrode 74 - 20Hz,)","(Electrode 80 - 18Hz,)","(Electrode 52 - 20Hz,)","(Electrode 43 - 19Hz,)","(Electrode 105 - 19Hz,)","(Electrode 5 - 18Hz,)","(Electrode 103 - 21Hz,)","(Electrode 69 - 21Hz,)","(Electrode 75 - 21Hz,)","(Electrode 46 - 20Hz,)","(Electrode 70 - 15Hz,)","(Electrode 28 - 21Hz,)","(Electrode 78 - 23Hz,)","(Electrode 74 - 24Hz,)","(Electrode 50 - 19Hz,)","(Electrode 38 - 19Hz,)","(Electrode 81 - 19Hz,)","(Electrode 69 - 22Hz,)","(Electrode 47 - 18Hz,)","(Electrode 44 - 18Hz,)","(Electrode 61 - 20Hz,)","(Electrode 50 - 19Hz,)","(Electrode 53 - 16Hz,)","(Electrode 45 - 10Hz,)","(Electrode 52 - 15Hz,)","(Electrode 77 - 21Hz,)","(Electrode 62 - 22Hz,)","(Electrode 12 - 13Hz,)",...,"(Electrode 32 - 18Hz,)","(Electrode 59 - 20Hz,)","(Electrode 96 - 19Hz,)","(Electrode 67 - 19Hz,)","(Electrode 90 - 20Hz,)","(Electrode 47 - 19Hz,)","(Electrode 90 - 19Hz,)","(Electrode 69 - 20Hz,)","(Electrode 68 - 19Hz,)","(Electrode 64 - 18Hz,)","(Electrode 83 - 16Hz,)","(Electrode 75 - 19Hz,)","(Electrode 61 - 20Hz,)","(Electrode 51 - 17Hz,)","(Electrode 50 - 19Hz,)","(Electrode 79 - 20Hz,)","(Electrode 58 - 14Hz,)","(Electrode 100 - 9Hz,)","(Electrode 48 - 59Hz,)","(Electrode 74 - 37Hz,)","(Electrode 90 - 33Hz,)","(Electrode 65 - 18Hz,)","(Electrode 48 - 18Hz,)","(Electrode 51 - 17Hz,)","(Electrode 7 - 71Hz,)","(Electrode 57 - 20Hz,)","(Electrode 96 - 21Hz,)","(Electrode 44 - 18Hz,)","(Electrode 73 - 18Hz,)","(Electrode 53 - 19Hz,)","(Electrode 66 - 12Hz,)","(Electrode 32 - 19Hz,)","(Electrode 70 - 17Hz,)","(Electrode 96 - 40Hz,)","(Electrode 5 - 13Hz,)","(Electrode 53 - 19Hz,)","(Electrode 61 - 23Hz,)","(Electrode 52 - 21Hz,)","(Electrode 96 - 14Hz,)","(Electrode 90 - 20Hz,)"
0,-0.430488,-6.731675,-4.867600,3.466962,-2.565753,0.951381,0.479766,-2.068730,-0.683577,1.856075,1.968152,-1.293173,-0.309348,0.023570,0.392498,-1.048182,0.373093,-0.523378,0.566639,0.048809,0.891453,-0.423410,0.340094,0.225677,-1.217761,0.591754,-0.169153,-0.049330,0.619850,0.092936,0.463871,-0.061734,-0.825642,-0.237500,-0.054108,-0.975358,0.346841,0.166131,0.564759,-0.150454,...,0.193404,-0.022842,-0.258461,0.303247,-0.390491,0.213107,-0.113829,0.111510,0.179029,-0.307935,-0.037575,-0.297534,-0.108232,-0.127842,0.056654,0.198485,-0.245940,0.091644,-0.337840,0.263853,-0.063489,0.259677,0.082357,-0.481669,0.024867,0.083118,-0.168989,0.039983,-0.156093,-0.232707,0.108869,-0.109314,-0.527273,0.019883,0.104646,0.213855,0.183442,-0.052892,0.057435,-0.485763
1,-17.713054,4.533371,3.267931,1.522835,3.770185,0.107378,-0.066388,-0.169412,0.750597,-0.908189,1.446781,1.031082,1.012186,0.296521,-1.294421,-1.089174,0.408190,-1.911663,0.277802,0.334726,0.399368,-0.361170,-0.360488,0.234757,-0.384592,0.107894,0.767926,-0.299854,0.292538,-0.009939,-0.368145,0.480584,0.348879,-0.361656,-0.020477,0.359289,0.102560,-0.100110,0.259672,0.073540,...,-0.033912,0.148318,-0.354419,-0.171946,0.127179,0.174238,0.300080,-0.003168,0.347295,-0.030000,-0.046763,-0.226552,-0.028210,0.124624,0.199839,0.258869,0.345363,-0.197011,-0.285868,-0.297046,0.002065,-0.420913,-0.146939,0.203484,-0.199262,-0.111037,0.240463,-0.036174,0.152265,-0.307288,-0.203825,-0.037863,-0.388459,-0.250489,-0.479361,0.042396,0.186159,0.178691,-0.393640,0.139147
2,6.212079,-4.598784,-2.759346,-0.603485,1.982932,-1.403845,2.336635,-0.930353,2.045150,-0.435429,0.605799,0.506608,-0.988132,-0.425318,0.345815,0.772458,0.565836,-0.197192,-1.255781,1.603978,0.559727,-1.060139,1.269975,0.621092,-0.275001,0.277379,-0.208477,-0.623350,-0.244065,0.278680,-0.979904,-1.129354,0.398249,-0.671549,0.348280,-0.012998,0.049790,0.713386,-0.280496,0.049639,...,-0.390959,0.268886,-0.168773,0.299610,-0.272255,-0.025555,0.920960,-0.093207,0

In [ ]:
X['label'] = y
X

,"(Electrode 63 - 20Hz,)","(Electrode 56 - 16Hz,)","(Electrode 70 - 20Hz,)","(Electrode 75 - 23Hz,)","(Electrode 19 - 46Hz,)","(Electrode 86 - 59Hz,)","(Electrode 60 - 19Hz,)","(Electrode 60 - 16Hz,)","(Electrode 68 - 38Hz,)","(Electrode 57 - 18Hz,)","(Electrode 12 - 19Hz,)","(Electrode 61 - 17Hz,)","(Electrode 74 - 20Hz,)","(Electrode 80 - 18Hz,)","(Electrode 52 - 20Hz,)","(Electrode 43 - 19Hz,)","(Electrode 105 - 19Hz,)","(Electrode 5 - 18Hz,)","(Electrode 103 - 21Hz,)","(Electrode 69 - 21Hz,)","(Electrode 75 - 21Hz,)","(Electrode 46 - 20Hz,)","(Electrode 70 - 15Hz,)","(Electrode 28 - 21Hz,)","(Electrode 78 - 23Hz,)","(Electrode 74 - 24Hz,)","(Electrode 50 - 19Hz,)","(Electrode 38 - 19Hz,)","(Electrode 81 - 19Hz,)","(Electrode 69 - 22Hz,)","(Electrode 47 - 18Hz,)","(Electrode 44 - 18Hz,)","(Electrode 61 - 20Hz,)","(Electrode 50 - 19Hz,)","(Electrode 53 - 16Hz,)","(Electrode 45 - 10Hz,)","(Electrode 52 - 15Hz,)","(Electrode 77 - 21Hz,)","(Electrode 62 - 22Hz,)","(Electrode 12 - 13Hz,)",...,"(Electrode 59 - 20Hz,)","(Electrode 96 - 19Hz,)","(Electrode 67 - 19Hz,)","(Electrode 90 - 20Hz,)","(Electrode 47 - 19Hz,)","(Electrode 90 - 19Hz,)","(Electrode 69 - 20Hz,)","(Electrode 68 - 19Hz,)","(Electrode 64 - 18Hz,)","(Electrode 83 - 16Hz,)","(Electrode 75 - 19Hz,)","(Electrode 61 - 20Hz,)","(Electrode 51 - 17Hz,)","(Electrode 50 - 19Hz,)","(Electrode 79 - 20Hz,)","(Electrode 58 - 14Hz,)","(Electrode 100 - 9Hz,)","(Electrode 48 - 59Hz,)","(Electrode 74 - 37Hz,)","(Electrode 90 - 33Hz,)","(Electrode 65 - 18Hz,)","(Electrode 48 - 18Hz,)","(Electrode 51 - 17Hz,)","(Electrode 7 - 71Hz,)","(Electrode 57 - 20Hz,)","(Electrode 96 - 21Hz,)","(Electrode 44 - 18Hz,)","(Electrode 73 - 18Hz,)","(Electrode 53 - 19Hz,)","(Electrode 66 - 12Hz,)","(Electrode 32 - 19Hz,)","(Electrode 70 - 17Hz,)","(Electrode 96 - 40Hz,)","(Electrode 5 - 13Hz,)","(Electrode 53 - 19Hz,)","(Electrode 61 - 23Hz,)","(Electrode 52 - 21Hz,)","(Electrode 96 - 14Hz,)","(Electrode 90 - 20Hz,)",label
0,-0.430488,-6.731675,-4.867600,3.466962,-2.565753,0.951381,0.479766,-2.068730,-0.683577,1.856075,1.968152,-1.293173,-0.309348,0.023570,0.392498,-1.048182,0.373093,-0.523378,0.566639,0.048809,0.891453,-0.423410,0.340094,0.225677,-1.217761,0.591754,-0.169153,-0.049330,0.619850,0.092936,0.463871,-0.061734,-0.825642,-0.237500,-0.054108,-0.975358,0.346841,0.166131,0.564759,-0.150454,...,-0.022842,-0.258461,0.303247,-0.390491,0.213107,-0.113829,0.111510,0.179029,-0.307935,-0.037575,-0.297534,-0.108232,-0.127842,0.056654,0.198485,-0.245940,0.091644,-0.337840,0.263853,-0.063489,0.259677,0.082357,-0.481669,0.024867,0.083118,-0.168989,0.039983,-0.156093,-0.232707,0.108869,-0.109314,-0.527273,0.019883,0.104646,0.213855,0.183442,-0.052892,0.057435,-0.485763,No Diagnosis Given
1,-17.713054,4.533371,3.267931,1.522835,3.770185,0.107378,-0.066388,-0.169412,0.750597,-0.908189,1.446781,1.031082,1.012186,0.296521,-1.294421,-1.089174,0.408190,-1.911663,0.277802,0.334726,0.399368,-0.361170,-0.360488,0.234757,-0.384592,0.107894,0.767926,-0.299854,0.292538,-0.009939,-0.368145,0.480584,0.348879,-0.361656,-0.020477,0.359289,0.102560,-0.100110,0.259672,0.073540,...,0.148318,-0.354419,-0.171946,0.127179,0.174238,0.300080,-0.003168,0.347295,-0.030000,-0.046763,-0.226552,-0.028210,0.124624,0.199839,0.258869,0.345363,-0.197011,-0.285868,-0.297046,0.002065,-0.420913,-0.146939,0.203484,-0.199262,-0.111037,0.240463,-0.036174,0.152265,-0.307288,-0.203825,-0.037863,-0.388459,-0.250489,-0.479361,0.042396,0.186159,0.178691,-0.393640,0.139147,ADHD-Combined Type
2,6.212079,-4.598784,-2.759346,-0.603485,1.982932,-1.403845,2.336635,-0.930353,2.045150,-0.435429,0.605799,0.506608,-0.988132,-0.425318,0.345815,0.772458,0.565836,-0.197192,-1.255781,1.603978,0.559727,-1.060139,1.269975,0.621092,-0.275001,0.277379,-0.208477,-0.623350,-0.244065,0.278680,-0.979904,-1.129354,0.398249,-0.671549,0.348280,-0.012998,0.049790,0.713386,-0.280496,0.049639,...,0.268886,-0.168773,0.299610,-0.272255,-0.025555,0.920960,-0.093207,0.027174,0.

In [ ]:
X.to_csv('after_95pca_with6classes.csv')
